<a href="https://colab.research.google.com/github/HudcovicMarcel/grassland-management-from-aerial-imagery/blob/main/GMAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grassland Management from Aerial Imagery (GMAI)

This notebook serves as supplementary material for the [GMAI dataset and repository](https://github.com/HudcovicMarcel/grassland-management-from-aerial-imagery).  
It provides a **step-by-step demonstration** of the methodology used in the referenced study.  

The notebook can be run using the **Demo Data and Model** available from the Zenodo repository, which can be downloaded in the first step.

The demo data includes seven images and corresponding label files for each class in the dataset.  
**Note:** We do **not recommend training** the model with this small demo dataset.  
For actual training or reproducing results, please use the full `dataset.zip` from the repository.


**0.1. Demo data download**

Downloads demo data from Zenodo repository.

In [ ]:
import requests
import zipfile
import os
import shutil

demo_url = "https://zenodo.org/records/17350381/files/demo_data.zip?download=1"
model_url = "https://zenodo.org/records/17350381/files/YOLO11x_GMAI.pt?download=1"

output_dir = "GMAI"
demo_zip = "demo_data.zip"
model_file = os.path.join(output_dir, "YOLO11x_GMAI.pt")

r = requests.get(demo_url, stream=True)
with open(demo_zip, "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

os.makedirs(output_dir, exist_ok=True)
with zipfile.ZipFile(demo_zip, "r") as zip_ref:
    zip_ref.extractall(output_dir)

macosx_path = os.path.join(output_dir, "__MACOSX")
if os.path.exists(macosx_path):
    shutil.rmtree(macosx_path)

r = requests.get(model_url, stream=True)
with open(model_file, "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

print('Data downloaded')

**0.2. Install neccessary packages**

In [ ]:
!pip install rasterio
!pip install ultralytics

# **1. Dataset creation**





**1.1. Importing prerequsisites**

In [ ]:
import os
import rasterio
from rasterio.windows import Window
from collections import Counter
from PIL import Image
import cv2
import shutil
from collections import defaultdict
import random
import yaml


**1.2. Image tiling**

In [ ]:
img_path = '/content/GMAI/SAMPLE_ORTO' # Folder path with images
out_path = '/content/GMAI/SAMPLE_TILES' # Output folder for tiles

os.makedirs(out_path,exist_ok=True)

image_files = []

for dirpath, _, filenames in os.walk(img_path):
    for file in filenames:
        if file.lower().endswith(('.tif', '.tiff')): # Change according to your image file extension
            full_path = os.path.join(dirpath, file)
            image_files.append(full_path)
for image in image_files:
    with rasterio.open(image,"r+") as src:
        image_width = src.width
        image_height = src.height
        tiling_size = 1280  # Final size of image
        overlap = 0.1   # Overlap between images
        stride_x, stride_y = int(tiling_size * (1 - overlap)), int(tiling_size * (1 - overlap))

        tile_id=0

        for top in range(0, image_height, stride_y):
            for left in range(0, image_width, stride_x):
                if left + tiling_size > image_width:
                    left = image_width - tiling_size
                if top + tiling_size > image_height:
                    top = image_height - tiling_size
                window = Window(left, top, tiling_size, tiling_size)
                bounds = rasterio.windows.bounds(window, transform=src.transform)
                minX, minY, maxX, maxY = bounds
                tile_path = os.path.join(out_path, f"{os.path.basename(os.path.splitext(image)[0])}_{tile_id}.tif") #Tile name will contain name of image plus tile ID
                tile_id+=1
                tile_data = src.read(indexes=[1, 2, 3],window=window)
                with rasterio.open(
                                tile_path,
                                "w",
                                driver="GTiff",
                                height=tile_data.shape[1],
                                width=tile_data.shape[2],
                                count=3,
                                dtype=tile_data.dtype,
                                crs=src.crs,
                                transform=src.window_transform(window),
                            ) as dst:
                                dst.write(tile_data)

print("Created",(len(os.listdir(out_path))),'tiles from',(len(image_files)),'images')

**1.3. Data labeling**

All object annotations were created using the Label Studio graphical interface.  
We recommend running Label Studio on your local computer to label or review data, as it provides a user-friendly GUI and full control over the labeling process.
Further information on: https://labelstud.io/

**1.4. Data augmentation**

In [ ]:
input_images = '/content/GMAI/SAMPLE_DATASET/images'
input_labels = '/content/GMAI/SAMPLE_DATASET/labels'

output_images = '/content/GMAI/SAMPLE_DATASET/augmented/images'
output_labels = '/content/GMAI/SAMPLE_DATASET/augmented/labels'

os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

def read_yolo_labels(label_file):
    with open(label_file, 'r') as f:
        labels = [list(map(float, line.strip().split())) for line in f.readlines()]
    return labels

def write_yolo_labels(label_file, labels):
    with open(label_file, 'w') as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

# Flip horizontally
def flip_hor(image_path, labels):
    image = cv2.imread(image_path)
    flipped_image = cv2.flip(image, 1)
    flipped_labels = []
    for label in labels:
        class_id, x, y, w, h = label
        flipped_labels.append([int(class_id), 1 - x, y, w, h])
    return flipped_image, flipped_labels

# Flip vertically
def flip_ver(image_path, labels):
    image = cv2.imread(image_path)
    flipped_image = cv2.flip(image, 0)
    flipped_labels = []
    for label in labels:
        class_id, x, y, w, h = label
        flipped_labels.append([int(class_id), x, 1 - y, w, h])
    return flipped_image, flipped_labels

# Rotate 90° clockwise
def rotate_90(image_path, labels):
    image = cv2.imread(image_path)
    rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    rotated_labels = []
    for label in labels:
        class_id, x, y, w, h = label
        rotated_labels.append([int(class_id), 1 - y, x, h, w])
    return rotated_image, rotated_labels

# Rotate 90° counterclockwise
def rotate_90_ccw(image_path, labels):
    image = cv2.imread(image_path)
    rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    rotated_labels = []
    for label in labels:
        class_id, x, y, w, h = label
        rotated_labels.append([int(class_id), y, 1 - x, h, w])
    return rotated_image, rotated_labels

# Save augmented image and labels
def save_augmented(image, labels, base_name, suffix):
    output_image_path = os.path.join(output_images, f"{base_name}_{suffix}.tif")
    output_label_path = os.path.join(output_labels, f"{base_name}_{suffix}.txt")
    cv2.imwrite(output_image_path, image)
    write_yolo_labels(output_label_path, labels)

# Load original labels
class_counts = Counter()
for label_file in os.listdir(input_labels):
    if label_file.endswith(".txt"):
        with open(os.path.join(input_labels, label_file), "r") as f:
            lines = f.readlines()
            for line in lines:
                class_id = int(line.split()[0])
                class_counts[class_id] += 1


biggest_class_count = max(class_counts.values())
threshold_50 = biggest_class_count * 0.5
threshold_25 = biggest_class_count * 0.25

# Divides classes based on size
more_than_50 = [cls for cls, count in class_counts.items() if count > threshold_50]
between_25_and_50 = [cls for cls, count in class_counts.items() if threshold_25 < count <= threshold_50]
less_than_25 = [cls for cls, count in class_counts.items() if count <= threshold_25]
zero = [cls for cls, count in class_counts.items() if count == 0]

# Loop through label files
for label_file in os.listdir(input_labels):
    if not label_file.endswith(".txt"):
        continue

    label_path = os.path.join(input_labels, label_file)
    labels = read_yolo_labels(label_path)
    base_name = os.path.splitext(label_file)[0]

    # Copy original image and labels
    image_path = os.path.join(input_images, f"{base_name}.tif")
    shutil.copy(image_path, os.path.join(output_images, f"{base_name}.tif"))
    shutil.copy(label_path, os.path.join(output_labels, f"{base_name}.txt"))

    classes_in_image = [int(l[0]) for l in labels]

    # Apply augmentations based on class counts
    if any(cls in classes_in_image for cls in more_than_50 + zero):
        img, lbls = flip_hor(image_path, labels)
        save_augmented(img, lbls, base_name, "flipped")

    if any(cls in classes_in_image for cls in between_25_and_50):
        img, lbls = flip_hor(image_path, labels)
        save_augmented(img, lbls, base_name, "flipped_hor")
        img, lbls = flip_ver(image_path, labels)
        save_augmented(img, lbls, base_name, "flipped_ver")
        img, lbls = flip_hor(image_path, labels)
        lbls = [[l[0], 1 - l[1], l[2], l[3], l[4]] for l in lbls]
        save_augmented(img, lbls, base_name, "flipped_hor_ver")

    if any(cls in classes_in_image for cls in less_than_25):
        for func, suffix in [(flip_hor, "flipped_hor"), (flip_ver, "flipped_ver"),
                             (rotate_90, "rot_90"), (rotate_90_ccw, "rot_90_ccw")]:
            img, lbls = func(image_path, labels)
            save_augmented(img, lbls, base_name, suffix)

print("Created", len(os.listdir(output_images)), "augmented images from", len(os.listdir(input_images)), "original images.")

**1.5. Split dataset and create YAML file for training**

In [ ]:
images_folder = r"/content/GMAI/SAMPLE_DATASET/augmented/images"
labels_folder = r"/content/GMAI/SAMPLE_DATASET/augmented/labels"

output_folder = r"/content/GMAI/SAMPLE_DATASET/final_dataset"
os.makedirs(output_folder, exist_ok=True)

# Define split ratios
split_ratios = {
    "train": 0.6,
    "val": 0.2,
    "test": 0.2
}

# Organize files by class
class_samples = defaultdict(int)
class_files = defaultdict(list)
for label_file in os.listdir(labels_folder):
    if label_file.endswith(".txt"):
        with open(os.path.join(labels_folder, label_file), "r") as f:
            lines = f.readlines()
            for line in lines:
                class_id = int(line.split()[0])
                class_samples[class_id] += 1
            if lines:
                class_files[class_id].append(label_file)

# Create output directories
for split in split_ratios.keys():
    os.makedirs(os.path.join(output_folder, split, "labels"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, split, "images"), exist_ok=True)

# Dictionary to store class distributions after split
split_class_counts = {"train": Counter(), "val": Counter(), "test": Counter()}

# Split each class equally
for class_id, files in class_files.items():
    random.shuffle(files)

    train_split = int(len(files) * split_ratios["train"])
    val_split = int(len(files) * split_ratios["val"])

    train_files = files[:train_split]
    val_files = files[train_split:train_split + val_split]
    test_files = files[train_split + val_split:]

    for split, split_files in zip(["train", "val", "test"], [train_files, val_files, test_files]):
        sample_count = 0
        for file in split_files:
            with open(os.path.join(labels_folder, file), "r") as f:
                sample_count += len(f.readlines())
        split_class_counts[split][class_id] = sample_count

        for file in split_files:
            shutil.copy(os.path.join(labels_folder, file), os.path.join(output_folder, split, "labels", file))
            image_file = file.replace(".txt", ".tif")
            image_path = os.path.join(images_folder, image_file)
            shutil.copy(image_path, os.path.join(output_folder, split, "images", image_file))

class_names = {
    0: "Cow",
    1: "Haybale_round",
    2: "Haybale_square",
    3: "Haystack",
    4: "Horse",
    5: "Machinery",
    6: "Sheep"
}

# Create YAML file for training
yaml_data = {
    "train": os.path.abspath(os.path.join(output_folder, "train")),
    "val": os.path.abspath(os.path.join(output_folder, "val")),
    "test": os.path.abspath(os.path.join(output_folder, "test")),
    "nc": len(class_files),
    "names": [class_names.get(c, str(c)) for c in sorted(class_files.keys())]  # Class names
}

with open(os.path.join(output_folder, "dataset.yaml"), "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False)

# **2. Image quality assesment (IQA)**

**2.1. Importing prerequsisites**

In [ ]:
import glob
import pandas as pd
import random
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from skimage.exposure import match_histograms
import skimage
import matplotlib.pyplot as plt
import seaborn as sns

**2.2. Random selection of samples**


In [ ]:
images_path = '/content/GMAI/SAMPLE_DATASET/images' # Path to images
labels_path = '/content/GMAI/SAMPLE_DATASET/labels/' # Path to label files

txts = glob.glob(f"{labels_path}/**.txt", recursive=True)

all_classes = [[] for _ in range(7)]
samples = []

for file in txts:
    data = pd.read_csv(file, sep=" ", header=None)
    class_ids = data[0].unique()

    for class_num in class_ids:
        if 0 <= class_num < 7:
            all_classes[int(class_num)].append(file)

# Pick random samples for each class
for x in all_classes:
    sample = random.sample(x,1) # Adjust number of samples for each class
    samples.append(sample)

img_x = []
img_y = []
img_name = []

# Filter samples at the edges of images
for class_num in range(7):
    count = 0
    for file in samples[class_num]:
        data = pd.read_csv(file, sep=" ", header=None)
        data = data[data[0] == class_num]
        data = data.sample(n=1)
        y = int(data[1].iloc[0] * 1280 - 30)
        x = int(data[2].iloc[0] * 1280 - 30)
        if y > 60 and y < 1220:
            if x > 60 and x < 1220:
                img_y.append(y)
                img_x.append(x)
                img_name.append(os.path.join(images_path, os.path.basename(os.path.splitext(file)[0]+'.tif')))

**2.3. Resizing and IQA metrics calculation**


In [ ]:
def IQA(image,x,y):
    image = Image.open(image)
    image = image.convert("RGB")
    image = np.array(image)

    crop_size = 60

    cropped = tf.image.crop_to_bounding_box(image, x, y, crop_size, crop_size) # Crops image around sample
    resized_20 = tf.image.resize(cropped, [int(crop_size * 0.75), int(crop_size * 0.75)], method = 'area')/255 # Resize image to correspond GSD of 20cm
    resized_25 = tf.image.resize(cropped, [int(crop_size * 0.6), int(crop_size * 0.6)], method = 'area')/255 # Resize image to correspond GSD of 25cm

    methods = ['nearest','bicubic','lanczos3','lanczos5','gaussian'] # Tested interpolation methods

    metrics = []

    for method in methods:
        globals()[f'resized_20_{method}'] = tf.image.resize(resized_20, [int(crop_size), int(crop_size)], method = method)
        globals()[f'resized_25_{method}'] = tf.image.resize(resized_25, [int(crop_size), int(crop_size)], method = method)

        img=globals()[f'resized_20_{method}'].numpy()
        ref=tf.image.convert_image_dtype(cropped, dtype=tf.float32).numpy()

        # Resize images from 20cm back to 15cm and compute IQA metrics
        matched_20 = match_histograms(img, ref, channel_axis= -1)

        SSIM_20 = tf.image.ssim(cropped/255, globals()[f'resized_20_{method}'], max_val=1.0,filter_size=9,
                              filter_sigma=1, k1=0.01, k2=0.03)
        SSIM_20_matched = tf.image.ssim(cropped/255, matched_20, max_val=1.0,filter_size=9,
                              filter_sigma=1, k1=0.01, k2=0.03)
        PSNR_20 = tf.image.psnr(cropped/255, globals()[f'resized_20_{method}'], max_val = 1.0)
        PSNR_20_matched = tf.image.psnr(cropped/255, matched_20, max_val = 1.0)
        MSE_20 = skimage.metrics.mean_squared_error(ref,img)
        MSE_20_matched = skimage.metrics.mean_squared_error(ref,matched_20)
        MAE_20 = tf.reduce_mean(tf.abs(cropped/255-globals()[f'resized_20_{method}']))
        MAE_20_matched = tf.reduce_mean(tf.abs(cropped/255-matched_20))

        # Resize images from 25cm back to 15cm and compute IQA metrics
        img=globals()[f'resized_25_{method}'].numpy()
        matched_25 = match_histograms(img, ref, channel_axis= -1)

        SSIM_25 = tf.image.ssim(cropped/255, globals()[f'resized_25_{method}'], max_val=1.0,filter_size=9,
                              filter_sigma=1, k1=0.01, k2=0.03)
        SSIM_25_matched = tf.image.ssim(cropped/255, matched_25, max_val=1.0,filter_size=9,
                              filter_sigma=1, k1=0.01, k2=0.03)
        PSNR_25 = tf.image.psnr(cropped/255, globals()[f'resized_25_{method}'], max_val = 1.0)
        PSNR_25_matched = tf.image.psnr(cropped/255, matched_25, max_val = 1.0)
        MSE_25 = skimage.metrics.mean_squared_error(ref,img)
        MSE_25_matched = skimage.metrics.mean_squared_error(ref,matched_25)
        MAE_25 = tf.reduce_mean(tf.abs(cropped/255-globals()[f'resized_25_{method}']))
        MAE_25_matched = tf.reduce_mean(tf.abs(cropped/255-matched_25))

        globals()[f'stat_{method}_20'] = [float(SSIM_20),float(PSNR_20),float(MSE_20),float(MAE_20)]
        globals()[f'stat_{method}_20_matched'] = [float(SSIM_20_matched),float(PSNR_20_matched),float(MSE_20_matched),float(MAE_20_matched)]
        globals()[f'stat_{method}_25'] = [float(SSIM_25),float(PSNR_25),float(MSE_25),float(MAE_25)]
        globals()[f'stat_{method}_25_matched'] = [float(SSIM_25_matched),float(PSNR_25_matched),float(MSE_25_matched),float(MAE_25_matched)]

        metrics.append(globals()[f'stat_{method}_20'])
        metrics.append(globals()[f'stat_{method}_20_matched'])
        metrics.append(globals()[f'stat_{method}_25'])
        metrics.append(globals()[f'stat_{method}_25_matched'])

        return metrics

all_metrics = []
for i in range(len(img_name)):
    metrics = IQA(img_name[i], img_x[i], img_y[i])
    all_metrics.extend(metrics)

methods = ['nearest', 'bicubic', 'lanczos3', 'lanczos5', 'gaussian']
matched = ['no', 'yes']

data = []

for i, m in enumerate(all_metrics):
    method_idx = (i // 4) % 5
    matched_flag = matched[i % 2]
    data.append({
        'method': methods[method_idx],
        'matched': matched_flag,
        'SSIM': m[0],
        'PSNR': m[1],
        'MSE': m[2],
        'MAE': m[3]
    })

df = pd.DataFrame(data)
df_long = df.melt(id_vars=['method', 'matched'],
                  var_name='metric', value_name='value')

summary = (
    df_long
    .groupby(['metric', 'method', 'matched'])['value']
    .agg(['mean', 'std'])
    .reset_index()
)

summary['mean'] = summary['mean'].round(4)
summary['std'] = summary['std'].round(4)

print(summary)

# **3. YOLO11 training**

**3.1. Importing prerequsisites**

In [ ]:
from ultralytics import YOLO
import torch
import os

**3.2. Checks CUDA availability**

In [ ]:
print(f"Is CUDA supported? {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name()}")

**3.3. Training setup**

In [ ]:
model_sizes = ["n", "s", "m", "l", "x"] #Model variants
optimizers = ["AdamW","SGD"] #Optimizers
pretrained_options = [True,False] #Determing if model is pretrained or not
batch_sizes = [8, 16, 32] #Batch sizes
data_path = "/content/GMAI/SAMPLE_DATASET/final_dataset/dataset.yaml" #Path to YAML file of dataset
project_name = "training"
epochs = 100 #Epoch number
img_size = 1280
if torch.cuda.is_available(): #If CUDA is available, chooses GPU for training
  device = [0]
else:
  device = 'cpu'

**3.4. Training loop**

In [ ]:
for size in model_sizes:
    for optimizer in optimizers:
        for pretrained in pretrained_options:
            for batch_size in batch_sizes:
                run_name = f"YOLO11_{size}_{optimizer}_bs{batch_size}_{'pretrained' if pretrained else 'scratch'}"
                print(f"\nTraining YOLOv11{size} | {optimizer} | Pretrained: {pretrained} | Batch: {batch_size} ...")

                if pretrained:
                    model = YOLO(f"yolo11{size}.pt")
                else:
                    model = YOLO(f"yolo11{size}.pt")
                    model.model.reset_parameters()  # Removes pretrained weights

                results = model.train(
                    data=data_path,
                    epochs=epochs,
                    imgsz=img_size,
                    batch=batch_size,
                    device=device,
                    project=project_name,
                    name=run_name,
                    optimizer=optimizer,
                    augment=False,
                    val=True,
                    verbose=True
                )

                torch.cuda.empty_cache()

# **4. Inference with trained model**

**4.1. Importing prerequsisites**

In [ ]:
from ultralytics import YOLO
import torch
import os
from pathlib import Path
import rasterio
from rasterio.windows import Window
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import numpy as np
from rasterio.crs import CRS
from google.colab import files

**4.2. Checks CUDA availability**

In [ ]:
print(f"Is CUDA supported? {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name()}")

if torch.cuda.is_available(): #If CUDA is available, chooses GPU for predictions
  device = [0]
else:
  device = 'cpu'

**4.2. Find images for predictions**



In [ ]:
img_path = r"/content/GMAI/SAMPLE_ORTO"
out_path = r"/content/GMAI/predictions"

os.makedirs(out_path,exist_ok=True)

all_polygons, all_classes, all_confidences = [], [], []

tif_files = []

for dirpath, _, filenames in os.walk(img_path):
    for file in filenames:
        if file.lower().endswith(('.tif', '.tiff')):
            full_path = os.path.join(dirpath, file)
            tif_files.append(full_path)

print("Images found:",len(tif_files))

**4.3. Inference loop**

In [ ]:
for image in tif_files:

    image_folder = os.path.join(out_path, os.path.basename(os.path.splitext(image)[0]))
    os.makedirs(image_folder,exist_ok=True)

    tile_folder = os.path.join(image_folder, "tile")
    os.makedirs(tile_folder,exist_ok=True)

    #Tile
    with rasterio.open(image,"r+") as src:
        crs = src.crs
        image_width = src.width
        image_height = src.height
        tiling_size = 1280
        overlap = 0.1
        stride_x, stride_y = int(tiling_size * (1 - overlap)), int(tiling_size * (1 - overlap))

        tile_id=0

        for top in range(0, image_height, stride_y):
            for left in range(0, image_width, stride_x):
                if left + tiling_size > image_width:
                    left = image_width - tiling_size
                if top + tiling_size > image_height:
                    top = image_height - tiling_size
                window = Window(left, top, tiling_size, tiling_size)
                bounds = rasterio.windows.bounds(window, transform=src.transform)
                minX, minY, maxX, maxY = bounds
                tile_path = os.path.join(tile_folder, f"{os.path.basename(os.path.splitext(image)[0])}_{tile_id}.tif")
                tile_id+=1
                tile_data = src.read(indexes=[1, 2, 3],window=window)
                with rasterio.open(
                                tile_path,
                                "w",
                                driver="GTiff",
                                height=tile_data.shape[1],
                                width=tile_data.shape[2],
                                count=3,
                                dtype=tile_data.dtype,
                                crs=crs,
                                transform=src.window_transform(window),
                            ) as dst:
                                dst.write(tile_data)

    if tiling_size != 1280:
      for image in os.listdir(image_folder):
          image_path = os.path.join(image_folder,image)
          image = Image.open(image)
          image = image.convert("RGB")
          image = np.array(image)
          image = tf.image.resize(image, [1280, 1280], method = 'lanczos5')
          image = tf.cast(image, tf.uint8).numpy()
          tiff.imwrite(image_path, image)


    model = YOLO("/content/GMAI/YOLO11x_GMAI.pt")

    results = model.predict(
        source=tile_folder,
        conf=0.3,
        imgsz=1280,
        save_txt=True,
        save_conf=True,
        project = image_folder,
        name='predict')

    for result in results:
            image_path = Path(result.path)
            image_name = image_path.stem
            boxes = result.boxes

            if boxes is not None and len(boxes) > 0:
                result_path = os.path.join(image_folder, f"{image_name}.tif")
                result.save(filename=result_path)

                txt_filename = f"{image_name}.txt"
                txt_source_path = os.path.join(image_folder,"predict/labels", txt_filename)  # Default YOLO output folder
                txt_dest_path = os.path.join(image_folder,'labels', txt_filename)
                os.makedirs(os.path.dirname(txt_dest_path), exist_ok=True)
                if os.path.exists(txt_source_path):
                    shutil.move(txt_source_path, txt_dest_path)


shutil.rmtree(os.path.join(image_folder,"predict"))

**4.4. Shapefile creation**

In [ ]:
gdf = {}
if os.path.isdir(os.path.join(image_folder,'labels')):
  for file in os.listdir(os.path.join(image_folder,'labels')):
    df = pd.read_csv(os.path.join(image_folder,'labels',file), delimiter=' ', header=None, names=["class_id", "cx", "cy", "w", "h", "conf"])
    tile_path = os.path.join(tile_folder,f"{os.path.basename(os.path.splitext(file)[0])}.tif")

    with rasterio.open(tile_path) as src:
      transform = src.transform
      crs = src.crs
      img_w, img_h = src.width, src.height
      df[["cx", "w"]] *= img_w
      df[["cy", "h"]] *= img_h
      x_min, x_max = df["cx"] - df["w"] / 2, df["cx"] + df["w"] / 2
      y_min, y_max = df["cy"] - df["h"] / 2, df["cy"] + df["h"] / 2
      bbox_coords = np.column_stack([x_min, y_min, x_min, y_max, x_max, y_max, x_max, y_min, x_min, y_min])
      bbox_coords = bbox_coords.reshape(-1, 5, 2)
      transformed_coords = np.apply_along_axis(lambda pt: transform * tuple(pt), 2, bbox_coords)

      polygons = [Polygon(coords) for coords in transformed_coords]

      all_polygons.extend(polygons)
      all_classes.extend(df["class_id"].astype(int))
      all_confidences.extend(df["conf"])

gdf = gpd.GeoDataFrame(
  {"geometry": all_polygons, "class_id": all_classes, "confidence": all_confidences},
   crs=crs
    )

gdf.to_file(os.path.join(out_path, "predictions.shp"), driver="ESRI Shapefile")

**4.5. Download results**

In [ ]:
zip_filename = "predictions.zip"
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', out_path)
files.download(zip_filename)

print(f"Folder '{out_path}' compressed to '{zip_filename}' and downloaded.")